In [1]:
%reload_ext autoreload
%autoreload 2
import autohf as hf
import numpy as np
from matplotlib import pyplot as plt
import time
import autograd
import chemistry as chem
import autograd.numpy as anp
import openfermion
from pennylane import qchem
from tqdm import tqdm
import pennylane as qml
from autograd.differential_operators import make_jvp_reversemode as mjr

# Optimizing a Basis Set Using AutoHF

One of the interesting applications is building better basis sets. More specifically, when we choose a basis set for use in a VQE simulation, we can leave some of the implicit parameters that define the basis set "free", so that we may optimize them along with the circuit parameters.

In this demo, we will leave the exponents in the Gaussians used to define out primitive basis set as free parameters in a VQE simulation of the $H_2$ molecule. We begin by defining the molecule:

In [15]:
molecule = chem.H2()
R = molecule.optimal_coordinates
symbols = molecule.structure

We then define the corresponding basis set:

In [3]:
M1, M2 = hf.basis_set_params('sto-3g', symbols) # Generates default information for hydrogen
num_elecs = 2
charges = [1, 1]
R1, R2 = R[0:3], R[3:6]

basis_set = []
initial_exp = []

# Generates atomic orbitals using the default information
for func in M1:
    L, exp, coeff = func
    initial_exp.extend(exp)
    basis_set.append(hf.AtomicBasisFunction(L,  R=R1))
    
for func in M2:
    L, exp, coeff = func
    initial_exp.extend(exp)
    basis_set.append(hf.AtomicBasisFunction(L,  R=R2))

We can easily get the function needed to compute the one and two electron integrals:

In [8]:
core, active = qchem.active_space(2, 2, active_electrons=2, active_orbitals=2) # Prepares active space
elec_func_flat = hf.electron_integrals_flat(num_elecs, charges, basis_set, occupied=core, active=active) # Function to get electronic integrals

Next, we define methods which take a set of basis set parameters (in this case, exponents), and output the Hamiltonian and the gradient of the Hamiltonian:

In [12]:
# Defines some parameters
bohr_angs = 0.529177210903
wires = [0, 1, 2, 3]
new_coordinates = bohr_angs * np.array([R[0:3], R[3:6]])
geometry = list(zip(['H', 'H'], new_coordinates))

# Defines the nuclear energy
nuc_energy = hf.nuclear_energy(charges)([R1, R2])

# Defines the Hamiltonian and Hamiltonian gradient

def H(alpha):
    # Generates the molecular Hamiltonian
    integrals = elec_func_flat([R1, R2], [alpha[0:3]], [alpha[3:6]])
    
    one, two = integrals[1:5].reshape((2, 2)), integrals[5:].reshape((2, 2, 2, 2))
    return hf.build_h_from_integrals(geometry, one, two, nuc_energy, wires)

In [13]:
def dH(alpha, vec):
    # Generates derivative of molecular Hamiltonian
    derivative_func = lambda alpha : elec_func_flat([R1, R2], [alpha[0:3]], [alpha[3:6]])
    integrals_d = mjr(derivative_func)(alpha)(vec)
    
    one, two = integrals_d[1:5].reshape((2, 2)), integrals_d[5:].reshape((2, 2, 2, 2))
    return hf.build_h_from_integrals(geometry, one, two, 0.0, wires)

def grad_H(alpha):
    # Generates the gradient of the molecular Hamiltonian
    H1 = []
    for j in range(6):
        vec = np.array([1.0 if k == j else 0.0 for k in range(6)])
        H1.append(dH(alpha, vec))
    return H1

In [14]:
num_params = 1 # Number of variational parameters
qubits = 4 # Number of qubits

dev = qml.device('default.qubit', wires=qubits) # Defines the device used

# Creates the circuit used to construct the VQE function
def circuit(params, **kwargs):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=dev.wires)
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    
optimizers = (qml.GradientDescentOptimizer(stepsize=0.1), qml.GradientDescentOptimizer(stepsize=0.1))
initial_params = (np.array([0.2234384]), np.array(initial_exp))
steps = 20

# Performs VQE
energy, params, opt_exp = chem.analytic_geometry(H, grad_H, circuit, dev, optimizers, steps, initial_params, bar=True)

Energy = -1.1379034620472452, Geometry = 3.42501030837908:  10%| | 2


KeyboardInterrupt: 

In [15]:
def circuit(params, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=[0, 1, 2, 3])
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])

chem.vqe(molecule.hamiltonian()(R), circuit, dev, qml.GradientDescentOptimizer(stepsize=0.1), 200, np.array([0.0]), bar=True)

/Users/jackceroni/miniconda3/lib/python3.9/site-packages/pyscf/lib/misc.py:46: H5pyDeprecationWarning: Using default_file_mode other than 'r' is deprecated. Pass the mode to h5py.File() instead.
Energy = -1.1373060512183404: 100%|██| 200/200 [00:08<00:00, 23.08it/s]


(-1.1373060512183404, tensor([0.22348336], requires_grad=True))